In [7]:
from __future__ import absolute_import, division, print_function

import pathlib

import pandas as pd
import seaborn as sns

import tensorflow
print (tensorflow.__version__)



#from tensorflow import keras
#from tensorflow.keras import layers




1.4.0


In [3]:
column_names = ['/render/frame time/rendering/microseconds','ar_GI_diffuse_depth','ar_AA_samples','ar_GI_specular_samples','ar_GI_transmission_samples',
                'ar_GI_diffuse_samples', 'ar_GI_specular_depth', 'ar_bucket_size']


raw_dataset = pd.read_csv("output_data.csv", usecols=column_names, sep=",", skipinitialspace=True)

dataset = raw_dataset.copy()

#dataset = dataset.apply(pd.to_numeric) # convert all columns of DataFrame

dataset = dataset.astype("float64")

dataset.tail()


,ar_GI_specular_samples,/render/frame time/rendering/microseconds,ar_AA_samples,ar_bucket_size,ar_GI_diffuse_depth,ar_GI_diffuse_samples,ar_GI_specular_depth,ar_GI_transmission_samples
983,6.0,2.889439e+08,4.0,482.0,9.0,2.0,7.0,3.0
984,2.0,7.004720e+07,6.0,455.0,8.0,3.0,3.0,0.0
985,2.0,6.282489e+06,3.0,297.0,10.0,8.0,10.0,2.0
986,1.0,1.118134e+08,4.0,378.0,10.0,4.0,2.0,4.0
987,9.0,1.675580e+09,6.0,907.0,9.0,5.0,1.0,1.0


In [4]:
dataset.isna().sum()

ar_GI_specular_samples                       0
/render/frame time/rendering/microseconds    0
ar_AA_samples                                0
ar_bucket_size                               0
ar_GI_diffuse_depth                          0
ar_GI_diffuse_samples                        0
ar_GI_specular_depth                         0
ar_GI_transmission_samples                   0
dtype: int64

In [ ]:
#dataset.tail()

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

#train_dataset.tail()
#test_dataset

In [ ]:
sns.pairplot(train_dataset[column_names], diag_kind="kde")

In [ ]:
train_stats = train_dataset.describe()
#print (column_names[0])
train_stats.pop("/render/frame time/rendering/microseconds")
train_stats = train_stats.transpose()
train_stats

In [ ]:
#test_dataset.tail()
train_labels = train_dataset.pop("/render/frame time/rendering/microseconds")
test_labels = test_dataset.pop("/render/frame time/rendering/microseconds")
print (20*"#", train_labels)
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
def build_model():
  print (len(train_dataset.keys()))
  model = Sequential([
    tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()
model.summary()

In [2]:

example_batch = normed_train_data[:10]
example_batch
example_result = model.predict(example_batch)


NameError: name 'normed_train_data' is not defined

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 5000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])


In [3]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

NameError: name 'pd' is not defined

In [ ]:
import matplotlib.pyplot as plt



def plot_history(history):
  
  hist = pd.DataFrame(history.history)

  hist['epoch'] = history.epoch
  
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [/render/frame time/rendering/microsecond]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.legend()
  #plt.ylim([0,5])

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$/render/frame time/rendering/microsecond^2$]')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  plt.legend()
  #plt.ylim([0,20])
    
plot_history(history)